In [1]:
from scipy.integrate import odeint
from mpl_toolkits.mplot3d import Axes3D
from pylab import *
import random 
#!pip install import-ipynb
#import import_ipynb
import modelparameters as mpar
import numpy as np
from sympy import *
%load_ext Cython
import time



T 108.33078115826872


In [2]:
%%cython
cdef extern from "math.h":
    double sin(double)
    const double pi "M_PI"
cdef float aep = 1.1560
cdef float app = 0.8669
cdef float m_p = 1836.2
cdef float E_0 = 0.6
cdef float w_0 = 0.058
cdef int N = 3
cdef float T = 2 * pi * N /w_0


cpdef  float length2(list R, list r):
    return (R[0] - r[0]) ** 2 + (R[1] - r[1]) ** 2 + (R[2] - r[2]) ** 2
cpdef  float f_e(int i, list R_1, list R_2, list r):
    return (R_1[i] - r[i]) / (
                (length2(R_1,r) + aep ** 2) ** (3 / 2)) + (
                (R_2[i] - r[i])) /  (
                (length2(R_2,r) + aep ** 2) ** (3 / 2))
cpdef float f_p1(int i, list R_1, list R_2, list r) :
        return (-(R_1[i] - r[i]) / (
                (length2(R_1,r) + aep ** 2) ** (3 / 2)) + (R_1[i] - R_2[i]) / ((length2(R_1,R_2) + app ** 2) ** (3 / 2)))

cpdef float f_p2(int i, list R_1, list R_2, list r) :
        return (-(R_2[i] - r[i]) / (
                (length2(R_2,r) + aep ** 2) ** (3 / 2)) - (R_1[i] - R_2[i]) / ((length2(R_1,R_2) + app ** 2) ** (3 / 2)))
cpdef  list dif(  a, float t) :
    x, y, z, x_1, y_1, z_1, x_2, y_2, z_2, px, py, pz, p_1x, p_1y, p_1z, p_2x, p_2y, p_2z = a
    dadt = [px, py, pz,
            p_1x / m_p, p_1y / m_p, p_1z / m_p,
            p_2x / m_p, p_2y / m_p, p_2z / m_p,
            f_e(0,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]),
            f_e(1,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]),
            f_e(2,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]),
            f_p1(0,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]),
            f_p1(1,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]),
            f_p1(2,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]),
            f_p2(0,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]),
            f_p2(1,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]),
            f_p2(2,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z])]
    return dadt
cpdef float E(  t):
    E = E_0 * (sin(pi * t / T)) ** 2 * sin(w_0 * t)
    return E


cpdef list dif2(a, float t2):
    x, y, z, x_1, y_1, z_1, x_2, y_2, z_2, px, py, pz, p_1x, p_1y, p_1z, p_2x, p_2y, p_2z = a
    L = 100000
    dadt  = [px, py, pz,
            p_1x / m_p, p_1y / m_p, p_1z / m_p,
            p_2x / m_p, p_2y / m_p, p_2z / m_p,
            f_e(0,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]),
            f_e(1,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]),
            f_e(2,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]) - E(t2),
            f_p1(0,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]),
            f_p1(1,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]),
            f_p1(2,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]) + E(t2), 
            f_p2(0,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]),
            f_p2(1,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]),
            f_p2(2,[x_1, y_1, z_1],[x_2, y_2, z_2],[x, y, z]) + E(t2)] 
    return dadt

In [4]:

p = mpar.p
R_1 = [-mpar.dist/2, 0, 0] #начальные положения pr1,pr2,e
R_2 = [mpar.dist/2, 0, 0]
r = [0,0,0]

print (mpar.dist)
def a_0(i):
    a_0 = [0, 0, 0, -mpar.dist / 2, 0, 0, mpar.dist / 2, 0, 0, p[i][0], p[i][1], p[i][2], 0, 0, 0, 0, 0, 0]
    return a_0
solution = []
start = time.time()
#Решения 18 уравнений для n экспериментов
for i in range (mpar.n):
    solution.append(odeint(dif, a_0(i), mpar.t))

t2 = linspace(mpar.u, mpar.u + 325, 100000)


def a_00(i):
    a_00 = [solution[i][mpar.L - 1, 0], solution[i][mpar.L - 1, 1], solution[i][mpar.L - 1, 2], solution[i][mpar.L - 1, 3], solution[i][mpar.L - 1, 4], solution[i][mpar.L - 1, 5], solution[i][mpar.L - 1, 6], solution[i][mpar.L - 1, 7], solution[i][mpar.L - 1, 8], solution[i][mpar.L - 1, 9], solution[i][mpar.L - 1, 10], solution[i][mpar.L - 1, 11], solution[i][mpar.L - 1, 12], solution[i][mpar.L - 1, 13], solution[i][mpar.L - 1, 14], solution[i][mpar.L - 1, 15], solution[i][mpar.L - 1, 16], solution[i][mpar.L - 1, 17]]
    return a_00
solution2 = []

#print(solution[0][:, 0], solution[0][mpar.L-1, 0])

# Решения 18 уравнений для n экспериментов
for i in range(mpar.n):
    solution2.append(odeint(dif2, a_00(i), t2))

end = time.time()
vt = end-start
print('Время выполнения:',vt)
print('Среднее время выполнения одного эксперимента:',vt/mpar.nn)

1.5070016255625132
Время выполнения: 228.8739001750946
Среднее время выполнения одного эксперимента: 2.288739001750946


In [ ]:
%matplotlib qt
#Траектории частиц в первом эксперименте
plt.plot(mpar.t, solution[0][:, 0], '-b', marker='o', label='x_e(t)')
plt.plot(mpar.t, solution[0][:, 3], 'g', label='x_1(t)')
plt.plot(mpar.t, solution[0][:, 6], 'r', label='x_2(t)')
plt.legend(loc='best')
plt.xlabel('t')
plt.grid()
plt.show()

fig = plt.figure()
ax = Axes3D(fig)
y = solution[0][:, 1]
x = solution[0][:, 0]
z = solution[0][:, 2]
y_1 = solution[0][:, 4]
x_1 = solution[0][:, 3]
z_1 = solution[0][:, 5]
y_2 = solution[0][:, 7]
x_2 = solution[0][:, 6]
z_2 = solution[0][:, 8]
plt.plot(x,y,z,'--b',label = 'Электрон')
plt.plot(x_1, y_1, z_1,'g-',label = 'Протон_1')
plt.plot(x_2, y_2, z_2,'r',label = 'Протон_2')
plt.legend(loc='best')
ax.set_xlabel('X',fontsize=20) #rotation=60)
ax.set_ylabel('Y',fontsize=20)
ax.set_zlabel('Z', fontsize=20)
ax.yaxis._axinfo['label']['space_factor'] = 3.0

plt.show()

In [ ]:
%matplotlib qt
fig = plt.figure()
ax = Axes3D(fig)
y = solution2[0][:, 1]
x = solution2[0][:, 0]
z = solution2[0][:, 2]
y_1 = solution2[0][:, 4]
x_1 = solution2[0][:, 3]
z_1 = solution2[0][:, 5]
y_2 = solution2[0][:, 7]
x_2 = solution2[0][:, 6]
z_2 = solution2[0][:, 8]
ax.plot(x,y,z,'--b')
ax.plot(x_1, y_1, z_1,'g-')
ax.plot(x_2, y_2, z_2,'r')
plt.plot(x,y,z,'--b',label = 'Электрон')
plt.plot(x_1, y_1, z_1,'g-',label = 'Протон_1')
plt.plot(x_2, y_2, z_2,'r',label = 'Протон_2')
ax.set_xlabel('X',fontsize=20) #rotation=60)
ax.set_ylabel('Y',fontsize=20)
plt.legend(loc='best')
ax.set_zlabel('Z', fontsize=20)
ax.yaxis._axinfo['label']['space_factor'] = 3.0
plt.show()